In [ ]:
import os  # A module for interacting with os (make dirs, lists files, etc.)


from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# ONLY RUN TO MAKE BIN IMAGES

In [ ]:
# import cv2
# import os

# # Paths
# base_input_folder = '/content/drive/My Drive/Colab Notebooks/notes_project/testing_dataset_v2'
# base_output_folder = '/content/drive/My Drive/Colab Notebooks/notes_project/testing_dataset_v2'

# # Subfolders
# subfolders = ['images']

# # Process both 'train' and 'val'
# for subfolder in subfolders:
#     input_folder = os.path.join(base_input_folder, subfolder)
#     output_folder = os.path.join(base_output_folder, f"{subfolder}_bin")

#     os.makedirs(output_folder, exist_ok=True)

#     for filename in os.listdir(input_folder):
#         if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
#             img_path = os.path.join(input_folder, filename)
#             img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale

#             if img is None:
#                 print(f"⚠️ Skipping {img_path}, could not load.")
#                 continue

#             # Apply binary threshold
#             _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

#             # Save the binarized image
#             output_path = os.path.join(output_folder, filename)
#             cv2.imwrite(output_path, binary)

#     print(f"✅ Finished processing {subfolder} folder.")

# print('🎉 All images binarized and saved!')


✅ Finished processing images folder.
🎉 All images binarized and saved!


In [ ]:
!pip install ultralytics --upgrade -q
from ultralytics import YOLO
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyt

In [ ]:
import cv2
import numpy as np
import os
import glob
import shutil
from ultralytics import YOLO

# Define the rotation and binarization function
def rotate_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return image

    largest_contour = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest_contour)
    (center, (width, height), angle) = rect

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    abs_cos, abs_sin = abs(M[0, 0]), abs(M[0, 1])
    new_w = int(h * abs_sin + w * abs_cos)
    new_h = int(h * abs_cos + w * abs_sin)
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    rotated = cv2.warpAffine(image, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

    if rotated.shape[1] < rotated.shape[0]:
        rotated = cv2.rotate(rotated, cv2.ROTATE_90_CLOCKWISE)

    gray_rotated = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray_rotated, 127, 255, cv2.THRESH_BINARY)
    binary_rgb = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    return binary_rgb

# Load model
model = YOLO("/content/drive/My Drive/Colab Notebooks/notes_project/final_run/final.pt")

# Input and temporary preprocessing folder
image_dir = '/content/drive/My Drive/Colab Notebooks/notes_project/final_run/test'
temp_preprocessed_dir = '/content/preprocessed_images'

# Clean and create temp folder
if os.path.exists(temp_preprocessed_dir):
    shutil.rmtree(temp_preprocessed_dir)
os.makedirs(temp_preprocessed_dir, exist_ok=True)

# Preprocess and save rotated images
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
preprocessed_paths = []

for path in image_paths:
    image = cv2.imread(path)
    if image is None:
        print(f"⚠️ Could not read image: {path}")
        continue

    processed = rotate_image(image)
    filename = os.path.basename(path)
    save_path = os.path.join(temp_preprocessed_dir, filename)
    cv2.imwrite(save_path, processed)
    preprocessed_paths.append(save_path)

# Run inference
results = model.predict(source=preprocessed_paths, save=True, save_crop=True, imgsz=640, conf=0.25)
print(f"✅ Inference done on {len(preprocessed_paths)} images.")

# Handle predictions
detect_dir = "runs/detect"
predict_folders = sorted([f for f in os.listdir(detect_dir) if f.startswith("predict")],
                         key=lambda x: os.path.getctime(os.path.join(detect_dir, x)))
latest_predict_folder = os.path.join(detect_dir, predict_folders[-1])
print(f"📁 Latest prediction folder: {latest_predict_folder}")

# Define destination
dst = "/content/drive/My Drive/Colab Notebooks/notes_project/final_run/clean_set_predict_v1"

# Remove and move
if os.path.exists(dst):
    shutil.rmtree(dst)
shutil.move(latest_predict_folder, dst)
print(f"✅ Results moved to {dst}")



0: 640x640 1 5000_real, 4.4ms
1: 640x640 1 5000_real, 4.4ms
2: 640x640 1 5000_real, 4.4ms
3: 640x640 1 5000_real, 4.4ms
4: 640x640 1 5000_real, 4.4ms
5: 640x640 1 5000_real, 4.4ms
6: 640x640 1 5000_real, 4.4ms
7: 640x640 1 5000_real, 4.4ms
8: 640x640 1 500_real, 1 5000_real, 4.4ms
9: 640x640 1 5000_real, 4.4ms
10: 640x640 1 5000_real, 4.4ms
11: 640x640 1 5000_real, 4.4ms
12: 640x640 1 5000_real, 4.4ms
13: 640x640 1 5000_real, 4.4ms
14: 640x640 1 5000_real, 4.4ms
15: 640x640 1 5000_real, 4.4ms
16: 640x640 1 5000_real, 4.4ms
17: 640x640 1 5000_real, 4.4ms
18: 640x640 1 5000_real, 4.4ms
19: 640x640 1 5000_real, 4.4ms
20: 640x640 1 1000_real, 4.4ms
21: 640x640 1 1000_real, 4.4ms
22: 640x640 1 1000_real, 4.4ms
23: 640x640 1 1000_real, 4.4ms
24: 640x640 1 1000_real, 4.4ms
25: 640x640 1 1000_real, 4.4ms
26: 640x640 1 1000_real, 4.4ms
27: 640x640 1 1000_real, 4.4ms
28: 640x640 1 1000_real, 4.4ms
29: 640x640 1 1000_real, 4.4ms
30: 640x640 (no detections), 4.4ms
31: 640x640 1 500_real, 4.4ms
32